In [ ]:
import tensorflow as tf
!pip install silence_tensorflow
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import numpy as np
from sklearn.datasets import fetch_openml
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, Concatenate, BatchNormalization, IntegerLookup, StringLookup
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Load dataset
credit_data = fetch_openml(name='credit-g', version=1, as_frame=True)
X = credit_data.data
y = credit_data.target.map({'good': 1, 'bad': 0}).values

# Define feature columns
discrete_features = ['installment_commitment', 'residence_since', 'num_dependents', 'existing_credits']
categorical_features = X.select_dtypes(exclude='number').columns.tolist()
continuous_features = ['duration', 'credit_amount']

# Create TensorFlow datasets with 80/10/10 split
def create_tf_datasets(X, y, train_size=0.8, val_size=0.1, batch_size=128, seed=None):
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    dataset = dataset.shuffle(buffer_size=len(X), seed=seed)  # Configurable seed
    n = len(X)
    train_size = int(n * train_size)
    val_size = int(n * val_size)
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size + val_size)
    return (
        train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )

# Try different seeds (None for unseeded, or specific values)
seed = 2025  # Change to None for randomness, or test 0, 1, 42, etc.
train_dataset_raw, val_dataset_raw, test_dataset_raw = create_tf_datasets(X, y, seed=seed)

# Adapt lookup layers
def adapt_preprocessing_layers(dataset):
    ordinal_encoders = {col: IntegerLookup(output_mode='int', num_oov_indices=1) for col in discrete_features}
    categorical_encoders = {col: StringLookup(output_mode='int', num_oov_indices=1) for col in categorical_features}

    for batch in dataset:
        features, _ = batch
        for col in discrete_features:
            ordinal_encoders[col].adapt(features[col])
        for col in categorical_features:
            categorical_encoders[col].adapt(features[col])

    return ordinal_encoders, categorical_encoders

ordinal_encoders, categorical_encoders = adapt_preprocessing_layers(train_dataset_raw)

def log1p_with_shape(x):
    return tf.math.log1p(x)

def cast_to_float_with_shape(x):
    return tf.cast(x, tf.float32)


# Build model with preprocessing
def build_preprocessing_model():
    continuous_inputs = {col: Input(shape=(1,), dtype=tf.float32, name=f"{col}_input") for col in continuous_features}
    discrete_inputs = {col: Input(shape=(1,), dtype=tf.int32, name=f"{col}_input") for col in discrete_features}
    categorical_inputs = {col: Input(shape=(1,), dtype=tf.string, name=f"{col}_input") for col in categorical_features}

    processed_continuous = [
        BatchNormalization(momentum=0.1, epsilon=1e-5)(
            tf.keras.layers.Lambda(
                log1p_with_shape,
                output_shape=(1,),
                name=f'log1p_lambda_{col}'
            )(continuous_inputs[col])
        ) for col in continuous_features
    ]


    processed_discrete = [
        tf.keras.layers.Lambda(
            lambda x: cast_to_float_with_shape(ordinal_encoders[col](x)),
            output_shape=(1,),
            name=f'cast_lambda_{col}'
        )(discrete_inputs[col])
        for col in discrete_features
    ]

    embedding_size = 8
    embedded_features = [
        Flatten()(Embedding(input_dim=categorical_encoders[col].vocabulary_size(), output_dim=embedding_size)(
            categorical_encoders[col](categorical_inputs[col])
        )) for col in categorical_features
    ]

    all_features = Concatenate()(processed_continuous + processed_discrete + embedded_features)
    return continuous_inputs, discrete_inputs, categorical_inputs, all_features

# Build full model
continuous_inputs, discrete_inputs, categorical_inputs, processed_features = build_preprocessing_model()
x = Dense(128, activation='relu', kernel_initializer='he_normal')(processed_features)
x = tf.keras.layers.Dropout(0.1)(x)
x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
x = tf.keras.layers.Dropout(0.1)(x)
output = Dense(1, activation='sigmoid')(x)

model_inputs = list(continuous_inputs.values()) + list(discrete_inputs.values()) + list(categorical_inputs.values())
model = Model(inputs=model_inputs, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

# Preprocess datasets
def preprocess_batch(features, labels):
    inputs = {
        **{f"{col}_input": features[col] for col in continuous_features},
        **{f"{col}_input": tf.cast(features[col], tf.int32) for col in discrete_features},
        **{f"{col}_input": features[col] for col in categorical_features}
    }
    return inputs, labels

train_dataset = train_dataset_raw.map(preprocess_batch).cache()
val_dataset = val_dataset_raw.map(preprocess_batch).cache()
test_dataset = test_dataset_raw.map(preprocess_batch).cache()

# Callbacks
callbacks = [EarlyStopping(patience=15,
                           restore_best_weights=True,
                           monitor='val_loss'),
            ReduceLROnPlateau(monitor='val_loss',
                              fact=0.5,
                              patience=15,
                              min_lr=1e-6),
            ModelCheckpoint('best_logistic_credit_model_tf.keras',
                            monitor='val_loss',
                            save_best_only=True)
            ]

# Train
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=callbacks
)

# Evaluate
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test loss: {test_loss} - Test accuracy: {test_acc}")

# Save model
custom_objects = {
    'log1p_with_shape': log1p_with_shape,
    'cast_to_float_with_shape': cast_to_float_with_shape
}

model.save("logistic_credit_model_tf.keras")

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 428ms/step - accuracy: 0.4428 - loss: 0.7335 - val_accuracy: 0.6600 - val_loss: 0.6626 - learning_rate: 0.0010
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7008 - loss: 0.6151 - val_accuracy: 0.6600 - val_loss: 0.6729 - learning_rate: 0.0010
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.7021 - loss: 0.6007 - val_accuracy: 0.6600 - val_loss: 0.6517 - learning_rate: 0.0010
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.7093 - loss: 0.5816 - val_accuracy: 0.6700 - val_loss: 0.6377 - learning_rate: 0.0010
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.7152 - loss: 0.5720 - val_accuracy: 0.6500 - val_loss: 0.6274 - learning_rate: 0.0010
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.7202 - loss: 0.5609 - val_accuracy: 0.6800 - val_loss: 0.6145 - learning_rate: 0.0010
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7347 - loss: 0.5451 - val_accuracy: 0.680

In [7]:
tf.keras.models.load_model("credit_model_tf_large.keras",
                           custom_objects=custom_objects,
                           safe_mode=False)

<Functional name=functional_2, built=True>

In [ ]:
#For large dataset

import tensorflow as tf
from sklearn.datasets import fetch_openml
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, Concatenate, BatchNormalization, IntegerLookup, StringLookup
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Placeholder: Replace with your huge dataset loading logic
# For testing, we'll use credit-g; for production, use file-based loading below
credit_data = fetch_openml(name='credit-g', version=1, as_frame=True)
X = credit_data.data
y = credit_data.target.map({'good': 1, 'bad': 0}).values

# Define feature columns
discrete_num = ['installment_commitment', 'residence_since', 'num_dependents', 'existing_credits']
categorical_cols = X.select_dtypes(exclude='number').columns.tolist()
con_num_features = ['duration', 'credit_amount']

# Scalable dataset loading for huge datasets (uncomment and customize for your files)
"""
def create_tf_datasets(file_pattern, total_samples, train_size=0.8, val_size=0.1, batch_size=128, seed=42):
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        column_names=X.columns.tolist() + ['label'],  # Replace with your column names
        label_name='label',
        shuffle=True,
        shuffle_seed=seed,
        num_epochs=1
    )
    train_size = int(train_size * total_samples)
    val_size = int(val_size * total_samples)
    test_size = total_samples - train_size - val_size
    train_dataset = dataset.take(train_size // batch_size)
    val_dataset = dataset.skip(train_size // batch_size).take(val_size // batch_size)
    test_dataset = dataset.skip((train_size + val_size) // batch_size)
    return train_dataset, val_dataset, test_dataset

# Example usage for huge dataset
total_samples = 1000000  # Replace with your dataset size
train_dataset_raw, val_dataset_raw, test_dataset_raw = create_tf_datasets(
    file_pattern="path/to/files/*.csv", total_samples=total_samples
)
"""

# For credit-g testing
def create_tf_datasets(X, y, train_size=0.8, val_size=0.1, batch_size=128, seed=2025):
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    dataset = dataset.shuffle(buffer_size=len(X), seed=seed)  # Seed for consistency
    n = len(X)
    train_size = int(n * train_size)
    val_size = int(n * val_size)
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size + val_size)
    return (
        train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )

train_dataset_raw, val_dataset_raw, test_dataset_raw = create_tf_datasets(X, y)

# Adapt lookup layers for huge datasets (limit to a subset)
def adapt_preprocessing_layers(dataset, max_samples=10000):
    ordinal_encoders = {col: IntegerLookup(output_mode='int', num_oov_indices=1) for col in discrete_num}
    categorical_encoders = {col: StringLookup(output_mode='int', num_oov_indices=1) for col in categorical_cols}
    
    # Take a subset for adaptation (e.g., ~10k samples)
    sample_dataset = dataset.take(max_samples // 128)  # Adjust based on batch size
    for batch in sample_dataset:
        features, _ = batch
        for col in discrete_num:
            ordinal_encoders[col].adapt(features[col])
        for col in categorical_cols:
            categorical_encoders[col].adapt(features[col])
    
    return ordinal_encoders, categorical_encoders

ordinal_encoders, categorical_encoders = adapt_preprocessing_layers(train_dataset_raw)

# Build model with preprocessing
def build_preprocessing_model():
    continuous_inputs = {col: Input(shape=(1,), dtype=tf.float32, name=f"{col}_input") for col in con_num_features}
    discrete_inputs = {col: Input(shape=(1,), dtype=tf.int32, name=f"{col}_input") for col in discrete_num}
    categorical_inputs = {col: Input(shape=(1,), dtype=tf.string, name=f"{col}_input") for col in categorical_cols}

    processed_continuous = [
        BatchNormalization(momentum=0.1, epsilon=1e-5)(
            tf.keras.layers.Lambda(lambda x: tf.math.log1p(x))(continuous_inputs[col])
        ) for col in con_num_features
    ]

    processed_discrete = [
        tf.keras.layers.Lambda(lambda x: tf.cast(ordinal_encoders[col](x), tf.float32))(discrete_inputs[col])
        for col in discrete_num
    ]

    embedding_size = 8
    embedded_features = [
        Flatten()(Embedding(input_dim=categorical_encoders[col].vocabulary_size(), output_dim=embedding_size)(
            categorical_encoders[col](categorical_inputs[col])
        )) for col in categorical_cols
    ]

    all_features = Concatenate()(processed_continuous + processed_discrete + embedded_features)
    return continuous_inputs, discrete_inputs, categorical_inputs, all_features

# Build full model
continuous_inputs, discrete_inputs, categorical_inputs, processed_features = build_preprocessing_model()
x = Dense(128, activation='relu')(processed_features)
x = tf.keras.layers.Dropout(0.1)(x)
x = Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
output = Dense(1, activation='sigmoid')(x)

model_inputs = list(continuous_inputs.values()) + list(discrete_inputs.values()) + list(categorical_inputs.values())
model = Model(inputs=model_inputs, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Preprocess datasets
def preprocess_batch(features, labels):
    inputs = {
        **{f"{col}_input": features[col] for col in con_num_features},
        **{f"{col}_input": tf.cast(features[col], tf.int32) for col in discrete_num},
        **{f"{col}_input": features[col] for col in categorical_cols}
    }
    return inputs, labels

# No caching for huge datasets (enable for small datasets like credit-g)
train_dataset = train_dataset_raw.map(preprocess_batch)  # .cache() optional for small data
val_dataset = val_dataset_raw.map(preprocess_batch)      # .cache() optional
test_dataset = test_dataset_raw.map(preprocess_batch)    # .cache() optional

# Callbacks
callbacks = [EarlyStopping(patience=10, restore_best_weights=True)]

# Train
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=callbacks
)

# Evaluate
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

# Save model
model.save("credit_model_tf_large.keras")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
import random

# Set seeds for reproducibility
def set_seeds(seed=2025):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seeds(2025)  # Using the same seed as TensorFlow model

# Load dataset
credit_data = fetch_openml(name='credit-g', version=1, as_frame=True)
X = credit_data.data
y = credit_data.target.map({'good': 1, 'bad': 0}).values

# Define feature columns (same as in TensorFlow)
discrete_num = ['installment_commitment', 'residence_since', 'num_dependents', 'existing_credits']
categorical_cols = X.select_dtypes(exclude='number').columns.tolist()
con_num_features = ['duration', 'credit_amount']

# Split data using same proportions and seed as TensorFlow
def manual_split(X, y, train_size=0.8, val_size=0.1, seed=2025):
    np.random.seed(seed)
    n = len(X)
    train_idx = int(n * train_size)
    val_idx = int(n * (train_size + val_size))
    indices = np.random.permutation(n)
    train_indices = indices[:train_idx]
    val_indices = indices[train_idx:val_idx]
    test_indices = indices[val_idx:]
    return (
        X.iloc[train_indices], X.iloc[val_indices], X.iloc[test_indices],
        y[train_indices], y[val_indices], y[test_indices]
    )

X_train, X_val, X_test, y_train, y_val, y_test = manual_split(X, y)

# Calculate mean and std for continuous features (log-transformed)
continuous_stats = {}
for col in con_num_features:
    log_values = np.log1p(X_train[col].values)
    continuous_stats[col] = {'mean': log_values.mean(), 'std': log_values.std()}

# Custom Dataset Class
class CreditDataset(Dataset):
    def __init__(self, X, y, categorical_cols, discrete_num, con_num_features):
        self.X = X.reset_index(drop=True)
        self.y = y
        self.categorical_cols = categorical_cols
        self.discrete_num = discrete_num
        self.con_num_features = con_num_features

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features = {}
        for col in self.con_num_features:
            features[col] = torch.tensor(self.X.loc[idx, col], dtype=torch.float32).unsqueeze(0)
        
        for col in self.discrete_num:
            features[col] = torch.tensor(self.X.loc[idx, col], dtype=torch.int64).unsqueeze(0)
        
        for col in self.categorical_cols:
            features[col] = self.X.loc[idx, col]
        
        label = torch.tensor(self.y[idx], dtype=torch.float32)
        return features, label

# Create datasets
train_dataset = CreditDataset(X_train, y_train, categorical_cols, discrete_num, con_num_features)
val_dataset = CreditDataset(X_val, y_val, categorical_cols, discrete_num, con_num_features)
test_dataset = CreditDataset(X_test, y_test, categorical_cols, discrete_num, con_num_features)

# Create data loaders
batch_size = 128  # Same as TensorFlow
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Preprocessing Layers - aligning with TensorFlow implementation
class Preprocessor(nn.Module):
    def __init__(self, con_num_features, discrete_num, categorical_cols, train_dataset, continuous_stats):
        super(Preprocessor, self).__init__()
        self.con_num_features = con_num_features
        self.discrete_num = discrete_num
        self.categorical_cols = categorical_cols
        self.continuous_stats = continuous_stats

        # Discrete numerical features: Integer encoding (like IntegerLookup in TF)
        self.ordinal_encoders = {}
        for col in discrete_num:
            unique_values = sorted(train_dataset.X[col].unique())
            self.ordinal_encoders[col] = {val: idx + 1 for idx, val in enumerate(unique_values)}
            # +1 for 1-based indexing (0 reserved for OOV)
        
        # Categorical features: String lookup (like StringLookup in TF)
        self.string_lookups = {}
        self.embeddings = nn.ModuleDict()
        
        for col in categorical_cols:
            unique_values = sorted(train_dataset.X[col].unique())
            self.string_lookups[col] = {val: idx + 1 for idx, val in enumerate(unique_values)}
            # +1 for 1-based indexing (0 reserved for OOV)
            
            # Use embedding size of 8 to match TensorFlow
            self.embeddings[col] = nn.Embedding(
                num_embeddings=len(unique_values) + 1,  # +1 for OOV/padding
                embedding_dim=8  # Same as TensorFlow
            )

        # Batch normalization layers for continuous features
        self.batch_norms = nn.ModuleDict({
            col: nn.BatchNorm1d(1, momentum=0.1, eps=1e-5)  # Match TF params
            for col in con_num_features
        })

    def forward(self, features):
        processed_features = []

        # Continuous features: Log transform + BatchNorm (like TensorFlow)
        for col in self.con_num_features:
            # Log1p transform
            log_transformed = torch.log1p(features[col])
            # Apply batch normalization
            normalized = self.batch_norms[col](log_transformed)
            processed_features.append(normalized)

        # Discrete numerical features
        for col in self.discrete_num:
            # Map to integers with handling for OOV values
            indices = torch.tensor([
                self.ordinal_encoders[col].get(int(f.item()), 0) for f in features[col]
            ], dtype=torch.long, device=features[col].device)
            # Convert to float to match TensorFlow behavior
            processed_features.append(indices.unsqueeze(1).float())

        # Categorical features: lookup + embeddings
        for col in self.categorical_cols:
            # Get indices with OOV handling
            indices = torch.tensor([
                self.string_lookups[col].get(str(f), 0) for f in features[col]
            ], dtype=torch.long, device=next(self.embeddings[col].parameters()).device)
            
            # Apply embedding
            embedded = self.embeddings[col](indices)
            processed_features.append(embedded.view(embedded.size(0), -1))

        # Concatenate all features
        return torch.cat(processed_features, dim=1)

# Calculate input dimension
def calc_input_dim(con_num_features, discrete_num, categorical_cols, embedding_size=8):
    return (
        len(con_num_features) +  # Continuous features
        len(discrete_num) +      # Discrete features
        len(categorical_cols) * embedding_size  # Embedded categorical features
    )

# Build model to match TensorFlow architecture
class CreditModel(nn.Module):
    def __init__(self, preprocessor, input_dim):
        super(CreditModel, self).__init__()
        self.preprocessor = preprocessor
        
        # Match TensorFlow architecture
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.1)  # Match TF dropout rate
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.1)
        self.output_layer = nn.Linear(64, 1)
        
        # Initialize weights (helps match TF performance)
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
    
    def forward(self, features):
        x = self.preprocessor(features)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        output = torch.sigmoid(self.output_layer(x))
        return output

# Initialize preprocessor and model
input_dim = calc_input_dim(con_num_features, discrete_num, categorical_cols)
preprocessor = Preprocessor(con_num_features, discrete_num, categorical_cols, train_dataset, continuous_stats)
model = CreditModel(preprocessor, input_dim)

# Loss and optimizer (match TensorFlow params)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Match TF learning rate

# Early stopping implementation (similar to TF EarlyStopping)
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None
    
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict().copy()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict().copy()
            self.counter = 0

# Training function
def train_model(model, train_loader, val_loader, num_epochs=50):
    early_stopping = EarlyStopping(patience=10)
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for features, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(features).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for features, labels in val_loader:
                outputs = model(features).squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = correct / total
        val_losses.append(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            model.load_state_dict(early_stopping.best_model_state)
            break
    
    if not early_stopping.early_stop and early_stopping.best_model_state is not None:
        model.load_state_dict(early_stopping.best_model_state)
    
    return train_losses, val_losses

# Train the model
train_losses, val_losses = train_model(model, train_loader, val_loader)

# Evaluate on test set
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features).squeeze()
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save model
torch.save(model.state_dict(), "credit_model_pytorch.pt")